**About** : This notebook is used to infer models.

In [ ]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Initialization

### Imports

In [ ]:
import os
import torch

print(torch.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = "-1"

In [ ]:
import os
import re
import cv2
import sys
import glob
import json
import time
import torch
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.metrics import *

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

In [ ]:
from utils.logger import Config, upload_to_kaggle

from params import *
from data.preparation import *

from model_zoo.models import define_model
from utils.metrics import *
from utils.torch import load_model_weights
from utils.plots import plot_sample
from inference.main import uniform_soup

## Expes

In [ ]:
# EXP_FOLDER = "../logs/2023-03-16/42/"
# EXP_FOLDER = "../logs/2023-03-19/0/"
# EXP_FOLDER = "../logs/2023-03-21/0/"
EXP_FOLDER = "../logs/2023-03-21/13/"
EXP_FOLDER = "../logs/2023-03-22/10/"
EXP_FOLDER = "../logs/2023-03-22/16/"
EXP_FOLDER = "../logs/2023-03-24/13/"
EXP_FOLDER = "../logs/2023-03-24/25/"
EXP_FOLDER = "../logs/2023-03-24/28/"
EXP_FOLDER = "../logs/2023-03-29/5/"
# EXP_FOLDER = "../logs/2023-03-30/2/"
EXP_FOLDER = "../logs/2023-04-04/20/"
EXP_FOLDER = "../logs/2023-04-05/30/"
EXP_FOLDER = "../logs/2023-04-06/10/"
EXP_FOLDER = "../logs/2023-04-07/12/"

EXP_FOLDER = "../logs/2023-04-12/2/"
EXP_FOLDER = "../logs/2023-04-13/29/"

EXP_FOLDER = "../logs/2023-04-17/42/"  # 0.7265 / x5 0.7273 / MTx10 0.7274

EXP_FOLDER = "../logs/2023-04-18/10/"

In [ ]:
config = Config(json.load(open(EXP_FOLDER + "config.json", "r")))

In [ ]:
df = prepare_data(DATA_PATH, config.processed_folder)

In [ ]:
if "fold" not in df.columns:
    folds = pd.read_csv(config.folds_file)
    df = df.merge(folds, how="left", on=["participant_id", "sequence_id"])

In [ ]:
try:
    pred_oof = np.load(EXP_FOLDER + "pred_oof.npy")
    score = accuracy(df['target'], pred_oof)
    print(f"-> CV acc : {score:.4f}")
except:
    pass

## Inference

### Preprocessing

In [ ]:
ROWS_PER_FRAME = 543  # number of landmarks per frame

def load_relevant_data_subset(pq_path):
    df = pd.read_parquet(pq_path)
    n_frames = int(len(df) / ROWS_PER_FRAME)
    data = df[['x', 'y', 'z']].values.reshape(n_frames, ROWS_PER_FRAME, 3)
    return df, data.astype(np.float32)

In [ ]:
KEPT_LANDMARKS = [
    [468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488],  # left hand
    [522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542],  # right hand
    [10, 54, 67, 132, 150, 152, 162, 172, 176, 234, 284, 297, 361, 379, 389, 397, 400, 454],  # silhouette
    [13, 37, 40, 61, 78, 81, 84, 87, 88, 91, 191, 267, 270, 291, 308, 311, 314, 317, 318, 321, 415],  # lips
    [500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511], # arms
    [205, 425],  # cheeks
]
MAPPING = [i + 1 for i in range(len(KEPT_LANDMARKS))]

TO_AVG = [
    [466, 387, 385, 398, 263, 390, 374, 381, 362],  # left_eye
    [246, 160, 158, 173, 33, 163, 145, 154, 133],
    [383, 293, 296, 285],  # left_eyebrow
    [156, 63, 66, 55],  # right_eyebrow
    [1, 2, 98, 327, 168],  # nose
]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Preprocessing(nn.Module):
    def __init__(self, type_embed, max_len=50, model_max_len=50):
        super(Preprocessing, self).__init__()

        self.type_embed = torch.from_numpy(type_embed[None, :].astype(np.float32))
        self.type_embed = self.type_embed.repeat(1000, 1)

        self.landmark_embed = torch.tensor(np.arange(120)).float().unsqueeze(0) + 1
        self.landmark_embed = self.landmark_embed.repeat(1000, 1)
        
#         self.ids = torch.from_numpy(np.sort(np.concatenate(KEPT_LANDMARKS)))
        self.ids = torch.from_numpy(np.concatenate(KEPT_LANDMARKS))

        self.to_avg = [torch.tensor(avg) for avg in TO_AVG]

        self.hands = torch.tensor(
            [468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488] + 
            [522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542]
        )
        
        self.frames = torch.tensor(np.arange(1000) + 1)
        
        self.max_len = torch.tensor([max_len])
        self.model_max_len = torch.tensor([model_max_len])

    def filter_sign(self, x):
        hands = x[:, self.hands, 0]
        nan_prop = torch.isnan(hands).float().mean(-1)            
        x = x[torch.where(nan_prop < 1)[0]]

        length = self.frames[:x.size(0)].max().unsqueeze(0)
        sz = torch.cat([length, self.max_len]).max()
        
        divisor = (((sz - self.max_len) > 0) * (sz / self.max_len) + 1).int()
        ids = (self.frames[:x.size(0)] % divisor) == 0
        return x[ids]
    
    def forward(self, x):
        x = self.filter_sign(x)
        n_frames = x.shape[0]     
        
        avg_ids = []
        for ids in self.to_avg:
            avg_id = x[:, ids].mean(1).unsqueeze(1)  # , keepdims=True)
            avg_ids.append(avg_id)

        x = torch.cat([x[:, self.ids]] + avg_ids, 1)

        type_embed = self.type_embed[:n_frames]
        landmark_embed = self.landmark_embed[:n_frames, :x.shape[1]]
        
        # Normalize & fill nans
        nonan = x[~torch.isnan(x)].view(-1, x.shape[-1])
        x = x - nonan.mean(0)[None, None, :]
        x = x / nonan.std(0, unbiased=False)[None, None, :]
        x[torch.isnan(x)] = 0

        # Concat
        x = torch.cat([
            type_embed.unsqueeze(-1), x, landmark_embed.unsqueeze(-1)
        ], -1).transpose(1, 2)
        
        x = x[:self.model_max_len]
        
        return x

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


class PreprocessingTF(keras.Model):
    def __init__(self, type_embed, max_len=50, model_max_len=50):
        super(PreprocessingTF, self).__init__()

        self.type_embed = tf.convert_to_tensor(type_embed[None, :].astype(np.float32))
        self.type_embed = tf.repeat(self.type_embed, 1000, axis=0)

        self.landmark_embed = tf.range(100, dtype=tf.float32)[tf.newaxis, :] + 1
        self.landmark_embed = tf.repeat(self.landmark_embed, 100, axis=0)

        # self.ids = torch.from_numpy(np.sort(np.concatenate(KEPT_LANDMARKS)))
        self.ids = tf.convert_to_tensor(np.concatenate(KEPT_LANDMARKS))

        self.to_avg = [tf.convert_to_tensor(avg) for avg in TO_AVG]

        self.hands = tf.convert_to_tensor(
            [468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488] + 
            [522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542]
        )

        self.frames = tf.range(1000) + 1

        self.max_len = tf.constant(max_len, dtype=tf.int32)
        self.model_max_len = tf.constant(model_max_len, dtype=tf.int32)

    def filter_sign(self, x):
        hands = tf.gather(x, self.hands, axis=1)[:, :, 0]
        nan_prop = tf.reduce_mean(tf.cast(tf.math.is_nan(hands), dtype=tf.float32), axis=-1)
        nan_mask = nan_prop < 1
        x = tf.boolean_mask(x, nan_mask)

        length = tf.reduce_max(self.frames[:tf.shape(x)[0]])
        sz = tf.reduce_max(tf.stack([length, self.max_len], axis=0))

        divisor = tf.where(sz - self.max_len > 0, sz // self.max_len + 1, 1)
        ids = tf.math.floormod(self.frames[:tf.shape(x)[0]], divisor) == 0
        x = tf.boolean_mask(x, ids)

        return x
    
    def call(self, x):
        x = self.filter_sign(x)
#         n_frames = x.shape[0]
        n_frames = tf.reduce_max(self.frames[:tf.shape(x)[0]])

        avg_ids = []
        for ids in self.to_avg:
            avg_id = tf.math.reduce_mean(tf.gather(x, ids, axis=1), axis=1, keepdims=True)
            avg_ids.append(avg_id)

        x = tf.concat([tf.gather(x, self.ids, axis=1)] + avg_ids, axis=1)

        type_embed = self.type_embed[:n_frames]
        landmark_embed = self.landmark_embed[:n_frames]

        # Normalize & fill nans
        nonan = tf.boolean_mask(x, tf.math.logical_not(tf.math.is_nan(x)))
        nonan = tf.reshape(nonan, (-1, 3))
        mean = tf.math.reduce_mean(nonan, axis=0)[tf.newaxis, tf.newaxis, :]
        x = x - tf.math.reduce_mean(nonan, axis=0)[tf.newaxis, tf.newaxis, :]
        x = x / tf.math.reduce_std(nonan, axis=0)[tf.newaxis, tf.newaxis, :]
        
#         x = tf.where(tf.math.is_nan(x))
        x = tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)
        
        # Concat
        x = tf.concat([
            tf.expand_dims(type_embed, -1), x, tf.expand_dims(landmark_embed, -1)
        ], -1)
        x = tf.transpose(x, [0, 2, 1])

        x = x[:self.model_max_len]

        return x

In [ ]:
landmarks = np.concatenate(KEPT_LANDMARKS)
type_embed = np.zeros(1000)
start = 0
for subset, idx in zip(KEPT_LANDMARKS, MAPPING):
    print(subset, idx)
    type_embed[start: start + len(subset)] = idx
    start += len(subset)

type_embed = type_embed[type_embed > 0]

type_embed = np.concatenate([type_embed, np.array([idx] * len(TO_AVG))])

print("\nn_landmarks :", len(type_embed))

In [ ]:
prepro = Preprocessing(type_embed, max_len=config.max_len, model_max_len=config.max_len)

In [ ]:
inp = tf.keras.Input((543, 3), dtype=tf.float32, name="inputs")
prepro_tf = PreprocessingTF(type_embed, max_len=config.max_len, model_max_len=config.max_len)

In [ ]:
preds = []
times = []

# for i in tqdm(range(len(df['path']))):
for i in tqdm(range(100)):
    path = df['path'][i]
    name = f"{path.split('/')[-2]}_{path.split('/')[-1].split('.')[0]}.npy"

    pq, data = load_relevant_data_subset(path)
    
    x = torch.from_numpy(data)
    x_ = tf.constant(data)

    x_torch = prepro(x)
    x_tf = prepro_tf(x_)
    
    print(np.abs(x_tf.numpy() - x_torch.numpy()).max())

    break

### Model

In [ ]:
import onnx
import onnx_tf
import tensorflow as tf
import onnxruntime as rt
import tflite_runtime.interpreter as tflite

from onnx_tf.backend import prepare

In [ ]:
import torch.nn as nn
from torch.nn import LayerNorm

class DebertaV2Output(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.intermediate_size, config.output_size)
        self.LayerNorm = LayerNorm(config.output_size, config.layer_norm_eps)
        self.config = config

    def forward(self, hidden_states, input_tensor):
        hidden_states = self.dense(hidden_states + input_tensor)
        hidden_states = self.LayerNorm(hidden_states)
        return hidden_states

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoConfig
from model_zoo.deberta import DebertaV2Encoder
from model_zoo.utils import add_shift
    
class Model(nn.Module):
    """
    Model with an attention mechanism.
    """
    def __init__(
        self,
        type_embed,
        embed_dim=256,
        dense_dim=384,
        transfo_dim=768,
        transfo_layers=3,
        transfo_heads=1,
        num_classes=250,
        drop_rate=0,
        n_landmarks=100,
        max_len=50,
    ):
        """
        Constructor.

        Args:
            encoder (timm model): Encoder.
            num_classes (int, optional): Number of classes. Defaults to 1.
            num_classes_aux (int, optional): Number of aux classes. Defaults to 0.
            n_channels (int, optional): Number of image channels. Defaults to 3.
        """
        super().__init__()
        self.num_classes = num_classes
        self.num_classes_aux = 0
        self.transfo_heads = transfo_heads

        self.type_embed = nn.Embedding(9, embed_dim, padding_idx=0)
        self.landmark_embed = nn.Embedding(101, embed_dim, padding_idx=0)
        self.type_norm = nn.LayerNorm(embed_dim)
        self.landmark_norm = nn.LayerNorm(embed_dim)

#         self.pos_dense = nn.Linear(9, embed_dim)
        self.pos_cnn = nn.Sequential(
            nn.Conv1d(3, 8, kernel_size=5, padding=2, bias=False),
            nn.Conv1d(8, 16, kernel_size=5, padding=2, bias=False),
        )
        self.pos_dense = nn.Linear(19, embed_dim)
        
        self.dense = nn.Linear(3 * embed_dim, embed_dim)
        
        self.left_hand_mlp = nn.Sequential(
            nn.Linear(embed_dim * 21, dense_dim), 
            nn.BatchNorm1d(dense_dim),
            nn.Dropout(p=drop_rate),
            nn.Mish(),
        )

        self.right_hand_mlp = nn.Sequential(
            nn.Linear(embed_dim * 21 , dense_dim),
            nn.BatchNorm1d(dense_dim),
            nn.Dropout(p=drop_rate),
            nn.Mish(),
        )

        self.lips_mlp = nn.Sequential(
            nn.Linear(embed_dim * 21, dense_dim),
            nn.BatchNorm1d(dense_dim),
            nn.Dropout(p=drop_rate),
            nn.Mish(),
        )
        
        self.face_mlp = nn.Sequential(
            nn.Linear(embed_dim * 25, dense_dim),
            nn.BatchNorm1d(dense_dim),
            nn.Dropout(p=drop_rate),
            nn.Mish(),
        )
        
        self.full_mlp = nn.Sequential(
            nn.Linear(embed_dim * n_landmarks, dense_dim),
            nn.BatchNorm1d(dense_dim),
            nn.Dropout(p=drop_rate),
            nn.Mish(),
        )

        transfo_dim_ = transfo_dim
        if transfo_layers == 3:  # 512, 768, 1024 / 768
            delta = 256
            transfo_dim = 512
        else:  # 768, 768 
            delta = 0
        self.transfo_dim = transfo_dim

        self.landmark_mlp = nn.Sequential(
            nn.Linear(dense_dim * 4, transfo_dim),
            nn.BatchNorm1d(transfo_dim),
            nn.Dropout(p=drop_rate),
            nn.Mish(),
        )

        name = "microsoft/deberta-v3-base"

        config = AutoConfig.from_pretrained(name, output_hidden_states=True)
        config.hidden_size = transfo_dim
        config.intermediate_size = transfo_dim
        config.output_size = transfo_dim
        if transfo_layers >= 2:
            config.output_size = transfo_dim + delta
        config.num_hidden_layers = 1
        config.num_attention_heads = transfo_heads
        config.attention_probs_dropout_prob = drop_rate
        config.hidden_dropout_prob = drop_rate
        config.hidden_act = nn.Mish()  # "relu"
        config.max_relative_positions = max_len
        config.position_buckets = max_len
        config.max_len = max_len

        self.frame_transformer_1 = DebertaV2Encoder(config)
        self.frame_transformer_1.layer[0].output = DebertaV2Output(config)

        self.frame_transformer_2 = None
        if transfo_layers >= 2:
            config.hidden_size += delta
            config.intermediate_size += delta
            
            if transfo_layers >= 3 and transfo_dim_ == 1024:
                config.output_size += delta

            config.attention_probs_dropout_prob *= 2
            config.hidden_dropout_prob *= 2
            self.frame_transformer_2 = DebertaV2Encoder(config)
            self.frame_transformer_2.layer[0].output = DebertaV2Output(config)

        self.frame_transformer_3 = None
        if transfo_layers >= 3:
            if transfo_dim_ == 1024:
                config.hidden_size += delta
                config.intermediate_size += delta
                config.attention_probs_dropout_prob *= 2
                config.hidden_dropout_prob *= 2

            self.frame_transformer_3 = DebertaV2Encoder(config)
            self.frame_transformer_3.layer[0].output = DebertaV2Output(config)

        self.logits = nn.Linear(config.output_size, num_classes)

    
    def forward(self, x):
        """
        Forward function.

        Args:
            x (torch tensor [batch_size x c x h x w]): Input batch.
            return_fts (bool, Optional): Whether to return encoder features.

        Returns:
            torch tensor [batch_size x num_classes]: logits.
            torch tensor [batch_size x num_classes_aux]: logits aux.
            torch tensor [batch_size x num_features]: Encoder features, if return_fts.
        """
        x = x.unsqueeze(0)

        bs, n_frames, _, n_landmarks = x.size()    

        x_type = self.type_norm(self.type_embed(x[:, :, 0].long()))
        x_landmark = self.landmark_norm(self.landmark_embed(x[:, :, 4].long()))
        
        x_pos_ = x[:, :, 1:4].transpose(2, 3).contiguous()
        
        x_pos = x_pos_.transpose(1, 2).transpose(2, 3).contiguous().view(bs * n_landmarks, 3, -1)
        x_pos = self.pos_cnn(x_pos)
        x_pos = x_pos.view(bs, n_landmarks, 16, -1).transpose(2, 3).transpose(1, 2).contiguous()

        x_pos = torch.cat([x_pos_, x_pos], -1)

        x_pos = self.pos_dense(x_pos)

        fts = self.dense(torch.cat([x_type, x_landmark, x_pos], -1))

        n_fts = fts.size(-1)
        embed = x[:, :, 0].unsqueeze(1).view(-1).long()

        left_hand_fts = fts.view(-1, n_fts)[embed == 1].view(-1, 21 * n_fts)
        left_hand_fts = self.left_hand_mlp(left_hand_fts)

        right_hand_fts = fts.view(-1, n_fts)[embed == 2].view(-1, 21 * n_fts)
        right_hand_fts = self.right_hand_mlp(right_hand_fts)
        
        hand_fts = torch.stack([left_hand_fts, right_hand_fts], -1).amax(-1)

        lips_fts = fts.view(-1, n_fts)[embed == 4].view(-1, 21 * n_fts)
        lips_fts = self.lips_mlp(lips_fts)

        face_fts = fts.view(-1, n_fts)[(embed == 3) | (embed == 6)].view(-1, 25 * n_fts)
        face_fts = self.face_mlp(face_fts)

        fts = fts.view(-1, n_fts * n_landmarks)
    
        fts = self.full_mlp(fts)

        fts = torch.cat([fts, hand_fts, lips_fts, face_fts], -1)

        fts = self.landmark_mlp(fts)
        fts = fts.view(bs, -1, self.transfo_dim)

        fts = self.frame_transformer_1(fts).last_hidden_state
        if self.frame_transformer_2 is not None:
            fts = self.frame_transformer_2(fts).last_hidden_state
        if self.frame_transformer_3 is not None:
            fts = self.frame_transformer_3(fts).last_hidden_state

        fts = fts.mean(1) # / mask.sum(1)  # masked avg

        logits = self.logits(fts)

        return logits

In [ ]:
model = Model(
    type_embed,
    embed_dim=config.embed_dim,
    dense_dim=config.dense_dim,
    transfo_dim=config.transfo_dim,
    transfo_layers=config.transfo_layers,
    transfo_heads=config.transfo_heads,
    drop_rate=config.drop_rate,
    num_classes=config.num_classes,
    max_len=config.max_len,
).cpu().eval()

config.embed_dim, config.transfo_dim

In [ ]:
N_SOUP = 10
TEACHER = True

In [ ]:
if N_SOUP:
    if not TEACHER:
        weights = [EXP_FOLDER + f"{config.name}_fullfit_0_{ep}.pt" for ep in range(config.epochs - N_SOUP, config.epochs + 1)]
    else:
        weights = [EXP_FOLDER + f"{config.name}_teacher_fullfit_0_{ep}.pt" for ep in range(config.epochs - N_SOUP, config.epochs + 1)]
    print("-> Soup :", [w.split('/')[-1] for w in weights])
    model = uniform_soup(model, weights)
    
else:
    try:
    #     model = load_model_weights(model, EXP_FOLDER + f"{config.name}_fullfit_0.pt")
    #     model = load_model_weights(model, EXP_FOLDER + f"{config.name}_teacher_fullfit_0.pt")
        model = load_model_weights(model, EXP_FOLDER + f"{config.name}_0.pt")
    except: # FileNotFoundError:
        print('Not loading weights !')

In [ ]:
# model.pos_cnn = nn.Sequential(
#     DenseConv1d(3, 8, conv_weights=model.pos_cnn[0].weight.data),
#     DenseConv1d(8, 16, conv_weights=model.pos_cnn[1].weight.data),
# )

In [ ]:
# x = np.load('../input/processed_3_torch/16069_100015657.npy')
# x = torch.from_numpy(x).unsqueeze(0)
# y = model(x)

In [ ]:
df = df[df['fold'] == 0].reset_index(drop=True)
pred_val = np.load(EXP_FOLDER + "pred_val_0.npy")

In [ ]:
accuracy(df['target'], pred_val)

In [ ]:
preds = []
times = []

# for i in tqdm(range(len(df['path']))):
for i in tqdm(range(100)):
    path = df['path'][i]
    name = f"{path.split('/')[-2]}_{path.split('/')[-1].split('.')[0]}.npy"

    pq, data = load_relevant_data_subset(path)
    
    x = torch.from_numpy(data)

    t0 = time.time()
    x = prepro(x)
    y = model(x)
    preds.append(y.detach().cpu().numpy().flatten())
    t1 = time.time()
    
    times.append((t1 - t0) * 1000)
#     break

In [ ]:
# rel_pos = torch.from_numpy(np.load('rel_pos.npy').astype(np.int32))
# rel_pos.size()

In [ ]:
print(f'Runtime : {np.mean(times) :.1f}ms')

In [ ]:
preds = np.stack(preds)

In [ ]:
accuracy(df['target'].head(len(preds)), preds)

In [ ]:
accuracy(df['target'].head(len(preds)), pred_val[:len(preds)])

#### Nobuco

In [ ]:
import tensorflow_addons as tfa

import nobuco
from nobuco import ChannelOrder, ChannelOrderingStrategy

In [ ]:
@nobuco.converter(torch.nn.functional.mish, channel_ordering_strategy=ChannelOrderingStrategy.MINIMUM_TRANSPOSITIONS)
def mish(input: torch.Tensor, inplace: bool = False):
    return lambda input, inplace=False: tfa.activations.mish(input)

In [ ]:
@nobuco.converter(torch.Tensor.long, channel_ordering_strategy=ChannelOrderingStrategy.MINIMUM_TRANSPOSITIONS)
def long(input: torch.Tensor, inplace: bool = False):
    return lambda input, inplace=False: tf.cast(input, tf.int64)

In [ ]:
@nobuco.converter(torch.Tensor.amax, channel_ordering_strategy=ChannelOrderingStrategy.FORCE_PYTORCH_ORDER)
def amax(input: torch.Tensor, dim=None, keepdim=False):
    return lambda input, axis: tf.reduce_max(input, axis=dim)

In [ ]:
@nobuco.converter(torch.gather, channel_ordering_strategy=ChannelOrderingStrategy.MINIMUM_TRANSPOSITIONS)
def gather(input, dim, index):
    return lambda input, dim, index: tf.gather(input, index)

In [ ]:
inp = x.clone()
inp.size()

In [ ]:
keras_model = nobuco.pytorch_to_keras(
    model,
    args=[inp],
    input_shapes={inp: (None, 5, 100)},
    inputs_channel_order=ChannelOrder.PYTORCH,
    outputs_channel_order=ChannelOrder.TENSORFLOW
)

In [ ]:
# keras_model.summary()

In [ ]:
preds = []
times = []

# for i in tqdm(range(len(df['path']))):
for i in tqdm(range(100)):
    path = df['path'][i]
    name = f"{path.split('/')[-2]}_{path.split('/')[-1].split('.')[0]}.npy"

    pq, data = load_relevant_data_subset(path)
    x = torch.from_numpy(data)

    x = prepro_tf(x)
    y = keras_model(x)
    preds.append(y.numpy()[0])

#     break

In [ ]:
accuracy(df['target'].head(len(preds)), preds)

### Prepro + model

In [ ]:
class TFLiteModel(tf.keras.Model):
    def __init__(self, prepro, model):
        super(TFLiteModel, self).__init__()
        self.prepro = prepro
        self.model = model
    
    @tf.function(input_signature=[tf.TensorSpec(shape=[None, 543, 3], dtype=tf.float32, name='inputs')])
    def call(self, inputs=None):
        x = self.prepro(tf.cast(inputs, dtype=tf.float32))
        y = self.model(x)

        # Return a dictionary with the output tensor
#         return x
        return {'outputs': y}

In [ ]:
# inp = tf.keras.Input((543, 3), dtype=tf.float32, name="inputs")
prepro_tf = PreprocessingTF(type_embed, max_len=config.max_len, model_max_len=config.max_len)

In [ ]:
tflite_keras_model = TFLiteModel(prepro_tf, keras_model)

In [ ]:
preds = []

# for i in tqdm(range(len(df['path']))):
for i in tqdm(range(100)):
    path = df['path'][i]
    name = f"{path.split('/')[-2]}_{path.split('/')[-1].split('.')[0]}.npy"

    pq, data = load_relevant_data_subset(path)

    y = tflite_keras_model(data)
    
    preds.append(y['outputs'].numpy()[0])
#     break

In [ ]:
accuracy(df['target'].head(len(preds)), preds)

In [ ]:
tflite_keras_model.save(EXP_FOLDER + 'model_keras')

### TfLite

In [ ]:
# custom_objects = {'WeightLayer': WeightLayer}

# keras_model_restored = keras.models.load_model(model_path + '.h5', custom_objects=custom_objects)
# print('Model loaded')

# converter = TFLiteConverter.from_keras_model_file(model_path + '.h5', custom_objects=custom_objects)
# converter.target_ops = [tf.lite.OpsSet.SELECT_TF_OPS, tf.lite.OpsSet.TFLITE_BUILTINS]
# tflite_model = converter.convert()
# with open(model_path + '.tflite', 'wb') as f:
#     f.write(tflite_model)

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(EXP_FOLDER + "model_keras")

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

# converter.target_spec.supported_ops = [
#     tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
#     tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
# ]

tflite_model = converter.convert()

with open(EXP_FOLDER + 'model.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
interpreter = tflite.Interpreter(EXP_FOLDER + "model.tflite")

prediction_fn = interpreter.get_signature_runner("serving_default")

output = prediction_fn(inputs=data)
output['outputs'].max()

In [ ]:
preds = []
times = []
# for i in tqdm(range(len(df['path']))):
for i in tqdm(range(10)):
    path = df['path'][i]
    name = f"{path.split('/')[-2]}_{path.split('/')[-1].split('.')[0]}.npy"

    pq, data = load_relevant_data_subset(path)

    t0 = time.time()
    output = prediction_fn(inputs=data)
    t1 = time.time()

    preds.append(output['outputs'])
    times.append((t1 - t0) * 1000)
    
#     break

In [ ]:
accuracy(df['target'].head(len(preds)), preds)

In [ ]:
print(f'-> Runtime : {np.mean(times) :.1f}ms')

if np.mean(times) > 100:
    print("\n WARNING ! Runtime must be < 100 ms !")

### Size & upload

In [ ]:
size = os.path.getsize(EXP_FOLDER + 'model.tflite') / np.power(1024, 2)
print(f"-> Model size : {size:.3f} Mo")

assert size < 40, "Model size must be < 40 Mo !"

In [ ]:
upload_to_kaggle([EXP_FOLDER], "/workspace/datasets/islr_weights_1/", "ISLR Models", update_folders=False)

Done ! 